In [7]:
import json
import numpy as np

from drawing.draw_svg import poses

with open("../drawing/drawing_config.json", "r") as f:
    config = json.load(f)

T, u, v, normal, origin = np.array(config["T"]), np.array(config["u"]), np.array(config["v"]), np.array(config["normal"]), np.array(config["origin"])

print("poses: ", config["poses"])
print("origin: ", origin)
print("u: ", u)
print("v: ", v)
print("normal: ", normal)
print("")

poses:  [{'ee.x': 0.2332200934104722, 'ee.y': 0.09912764874314312, 'ee.z': -0.002705391021625453, 'ee.wx': 0.47178912893823455, 'ee.wy': -2.873226245896773, 'ee.wz': 0.10518815843955037, 'ee.gripper_pos': 71.81153533712428}, {'ee.x': 0.2547830989673974, 'ee.y': -0.03877770216455046, 'ee.z': -0.0035513430087482645, 'ee.wx': -0.46828931504575055, 'ee.wy': -2.8676909107422865, 'ee.wz': -0.0065986483239183855, 'ee.gripper_pos': 71.81153533712428}, {'ee.x': 0.17203382552817434, 'ee.y': 0.11859896561747936, 'ee.z': 0.01164885963368019, 'ee.wx': 0.8040650854367607, 'ee.wy': -2.6912687576291816, 'ee.wz': 0.2132967914969007, 'ee.gripper_pos': 71.81153533712428}, {'ee.x': 0.17673042928176463, 'ee.y': -0.05238917038012587, 'ee.z': 0.018527441423327295, 'ee.wx': -0.681169341452901, 'ee.wy': -2.6090230331645663, 'ee.wz': -0.06892237300419922, 'ee.gripper_pos': 71.81153533712428}, {'ee.x': 0.1316569507157193, 'ee.y': 0.1227049510394132, 'ee.z': -0.004077691470871378, 'ee.wx': 1.1685823207522272, 'ee

In [4]:
import math

#compute orienation relative to plane
R = np.column_stack((u, v, normal))
def rotmat_to_euler_xyz(R):
    sy = math.sqrt(R[0,0]**2 + R[1,0]**2)
    singular = sy < 1e-6
    if not singular:
        roll  = math.atan2(R[2,1], R[2,2])
        pitch = math.atan2(-R[2,0], sy)
        yaw   = math.atan2(R[1,0], R[0,0])
    else:
        roll  = math.atan2(-R[1,2], R[1,1])
        pitch = math.atan2(-R[2,0], sy)
        yaw   = 0.0
    return np.array([roll, pitch, yaw])

euler_angles = rotmat_to_euler_xyz(R)
print("euler angles (radians): ", euler_angles)
print("euler angles (degrees): ", np.degrees(euler_angles))

euler angles (radians):  [ 3.13074863 -0.00968482 -1.41552058]
euler angles (degrees):  [179.37868338  -0.5548994  -81.10335493]


In [8]:
#compare with pose data
# Plane rotation matrix
z_plane = R[:,2]  # tool z-axis

def rotvec_to_rotmat(rotvec):
    theta = np.linalg.norm(rotvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rotvec / theta
    K = np.array([[0, -k[2], k[1]],
                  [k[2], 0, -k[0]],
                  [-k[1], k[0], 0]])
    R = np.eye(3) + np.sin(theta) * K + (1 - np.cos(theta)) * (K @ K)
    return R

# For each sample
for s in poses:
    rotvec = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])
    R_sample = rotvec_to_rotmat(rotvec)
    z_sample = R_sample[:,2]

    dot_product = np.dot(z_plane, z_sample)
    angle_deg = np.arccos(np.clip(dot_product, -1.0, 1.0)) * 180 / np.pi
    print(f"Angle between plane z-axis and sample z-axis: {angle_deg:.2f}°")

Angle between plane z-axis and sample z-axis: 12.94°
Angle between plane z-axis and sample z-axis: 13.14°
Angle between plane z-axis and sample z-axis: 19.69°
Angle between plane z-axis and sample z-axis: 25.37°
Angle between plane z-axis and sample z-axis: 2.58°
Angle between plane z-axis and sample z-axis: 5.64°


In [9]:
def rotmat_to_rotvec(R):
    """Convert rotation matrix to rotation vector (axis * angle)."""
    theta = math.acos(np.clip((np.trace(R) - 1)/2, -1.0, 1.0))
    if abs(theta) < 1e-12:
        return np.zeros(3)  # zero rotation
    rx = (R[2,1] - R[1,2]) / (2*np.sin(theta))
    ry = (R[0,2] - R[2,0]) / (2*np.sin(theta))
    rz = (R[1,0] - R[0,1]) / (2*np.sin(theta))
    axis = np.array([rx, ry, rz])
    return axis * theta

for s in poses:
    rotvec_sample = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])
    # Compute rotation difference vector
    R_sample = rotvec_to_rotmat(rotvec_sample)
    R_diff = R.T @ R_sample
    rotvec_diff = rotmat_to_rotvec(R_diff)

    angle_diff_deg = np.linalg.norm(rotvec_diff) * 180 / np.pi
    print(f"Rotation difference between plane and sample: {angle_diff_deg:.2f}°")

Rotation difference between plane and sample: 81.07°
Rotation difference between plane and sample: 117.81°
Rotation difference between plane and sample: 68.18°
Rotation difference between plane and sample: 129.34°
Rotation difference between plane and sample: 54.51°
Rotation difference between plane and sample: 139.04°
